# MNIST Classification

This tutorial demonstrates how to use the `lava.lib.dl.netx` API to classify MNIST images, using a `lava.lib.dl.slayer` trained network. The classification is done on both: CPU (via $\texttt{Loihi2SimCfg}$) and Loihi-2 neuro-cores (via $\texttt{Loihi2HwCfg}$).

Note that this inference tutorial is part of the **end-to-end training** and **evaluation** tutorial: [mnist-on-loihi](https://github.com/R-Gaurav/mnist-on-loihi) -- it contains all the inference code explained here, as well as the `slayer` training code to obtain the trained network-weights (used here). In this tutorial, we will _not_ be training the `slayer` network, rather load the exported (trained) weights via `netx` and do inference on two backends: $\textsf{Loihi-2 Simulation}$ ($\texttt{Loihi2SimCfg}$) and $\textsf{Loihi-2 Hardware}$ ($\texttt{Loihi2HwCfg}$). The `slayer` training procedure is explained in the [accompanying tutorial](https://r-gaurav.github.io/2024/04/13/Lava-Tutorial-MNIST-Training-on-GPU-and-Evaluation-on-Loihi2.html) and is quite straight-forward. However, there are some tips-and-tricks to keep in mind while evaluating the trained `slayer` network via `netx`; and that's precisely the point of this tutorial. Without further ado, let's begin!

# Load the `slayer`-trained weights